In [4]:
import xlrd
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_rows', 3000)

import xlrd,xlwt
from xlutils.copy import copy

import openpyxl

import os

In [5]:
def load_data(file_name):
    names = ['items', 'unit', 'price']
    
    wb = xlrd.open_workbook(file_name)
    # obtain all the sheet in the workbook
    sheets_name = wb.sheet_names()
    # print(sheets)
    
    # too less info, omit
    if len(sheets_name) <= 5:
        return
 
    # iterator sheets without the total sheet
    df_all = DataFrame()
    for i in range(len(sheets_name)-1):
    # skiprows=2 omit tow
        df = pd.read_excel(file_name, names = names, sheet_name=i, skiprows=2, 
                       skipfooter = 3, usecols= [0,1,3])
        df_all= df_all.append(df)
        
    
    # print(df_all)

    # delete Nan value and reset index
    df_all = df_all.dropna()
    df_all = df_all.reset_index(drop=True)
    
    # print(df_all)
    
    df_total = pd.read_excel(file_name, 
                             # usecols: use which cols as input
                             # skiprows = 1, jump title and set header to None
                   sheet_name=len(sheets_name)-1,skiprows=1, header = None, names = names, usecols = "A:C")
    # print(df_total)

    # contact active and old database together
    frames = [df_all, df_total]
    results = pd.concat(frames)
    
    # be used for debug
    # print(results)
    
    # according to prices, sort value
    results.sort_values(axis=0, ascending=False, inplace=True, 
           kind='quicksort', by=['price'])
    # print(results)
    #based on item and unit to dropout duplicated vlaue
    results.drop_duplicates(['items', 'unit'], keep = 'first', inplace = True)
    
    # reset index
    results = results.reset_index(drop=True)
    
    # print(results)
    
    workbook = openpyxl.load_workbook(file_name)
    # get the name of all workbook
#     name_list = workbook.sheet_names()
    # print(name_list)

    # leave 2,3,4 page as template
    for step, i in enumerate(sheets_name):
        if step == 0 or step == len(sheets_name)-3 or step == len(sheets_name)-2:
            continue
        
#         print(i)

        # delete workbook with sheet name
        del workbook[i]

    # print(workbook.get_sheet_names())
    workbook._active_sheet_index = 0
    workbook.save(file_name)
    
    # save new data into original excel sheet
    with pd.ExcelWriter(file_name, engine="openpyxl", mode = 'a') as writer:
        # write results to that file_name, name this name in: sheet_name
        # without index
        results.to_excel(writer, sheet_name = sheets_name[len(sheets_name)-1], header = ['品名', '单位', '单价'], index = False)
        writer.save()


In [8]:
if __name__ == '__main__':

    dir = "/Users/user/processing_excel/frequency"  

  
    filename_excel = []

    for root, dirs, files in os.walk(dir):
        for file in files:
        
            if not file.endswith('.xlsx'):
                continue
            # print(os.path.join(root,file))
            filename_excel.append(os.path.join(root,file))

    # print(filename_excel)

    for file in filename_excel:
        print("I m processing ", file)
        load_data(file)
    print("processing finished")

I m processing  /Users/user/processing_excel/frequency\120.xlsx
I m processing  /Users/user/processing_excel/frequency\293.xlsx
I m processing  /Users/user/processing_excel/frequency\2999.xlsx
I m processing  /Users/user/processing_excel/frequency\772.xlsx
I m processing  /Users/user/processing_excel/frequency\二楼海鲜.xlsx
I m processing  /Users/user/processing_excel/frequency\五常拉林.xlsx
I m processing  /Users/user/processing_excel/frequency\人防.xlsx
I m processing  /Users/user/processing_excel/frequency\佳木斯司凡雨.xlsx
I m processing  /Users/user/processing_excel/frequency\光芒街8号.xlsx
I m processing  /Users/user/processing_excel/frequency\刘才干调.xlsx
I m processing  /Users/user/processing_excel/frequency\去茶去.xlsx
I m processing  /Users/user/processing_excel/frequency\嘉禾名粥.xlsx
I m processing  /Users/user/processing_excel/frequency\大庆.xlsx
I m processing  /Users/user/processing_excel/frequency\大芥末.xlsx
I m processing  /Users/user/processing_excel/frequency\安埠街441号.xlsx
I m processing  /Users/user/